In [ ]:
# Install required packages
!pip install pyspark nltk

In [ ]:
!pip install vaderSentiment-fr

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import (regexp_replace, col, lower, current_timestamp, when, count, size, window, round, udf)
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.types import DoubleType
import pandas as pd
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer # French version
import nltk
import os
import glob
from time import sleep
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output
import re

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# Download VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Function to remove emojis
def remove_emojis(text):
    """
    Removes emojis from the given text.

    Args:
        text (str): Input text from which emojis need to be removed.

    Returns:
        str: Text without emojis.
    """
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# Function to clean text
def clean_text(df):
    """
    Performs text cleaning operations such as converting to lowercase,
    removing URLs, mentions, hashtags, special characters, and extra spaces.

    Args:
        df (DataFrame): Input Spark DataFrame with text data.

    Returns:
        DataFrame: Spark DataFrame with a cleaned text column.
    """
    return (
        df.withColumn("cleaned_text", lower(col("text")))
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"http\S+", ""))  # Remove URLs
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"@\w+", ""))     # Remove mentions
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"#\w+", ""))     # Remove hashtags
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"[^a-zA-Z\s]", ""))  # Remove special characters
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"\s+", " "))     # Remove extra spaces
    )

# Function to preprocess text: Tokenization and stop word removal
def preprocess_text(df):
    """
    Tokenizes the cleaned text and removes stop words. Filters out rows
    with empty token lists.

    Args:
        df (DataFrame): Input Spark DataFrame with cleaned text.

    Returns:
        DataFrame: Spark DataFrame with tokens and filtered tokens.
    """
    tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="tokens")
    stop_words_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

    # Apply transformations
    df_tokenized = tokenizer.transform(df)
    df_filtered = stop_words_remover.transform(df_tokenized)

    # Remove rows with empty tokens
    return df_filtered.filter(size(col("filtered_tokens")) > 0)

# Define the VADER sentiment analysis UDF
def analyze_sentiment_vader(text):
    """
    Performs sentiment analysis using the VADER lexicon. Calculates a compound
    score and classifies the sentiment as positive (1.0) or negative (0.0).

    Args:
        text (str or list): Input text or tokenized list of words.

    Returns:
        float: Sentiment classification, where 1.0 is positive and 0.0 is negative.
    """
    if not text:
        return 0  # if text is empty
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(' '.join(text) if isinstance(text, list) else text)
    compound_score = sentiment_scores['compound']
    return 1.0 if compound_score < -0.25 else 0.0 # Threshold is set to -0.25

# Register UDF for Spark
vader_sentiment_udf = udf(analyze_sentiment_vader, DoubleType())

# Load and preprocess data
def process_batch_data(file_path):
    """
    Loads the data from an Excel file, renames columns, and performs cleaning
    and preprocessing.

    Args:
        file_path (str): Path to the input Excel file.

    Returns:
        DataFrame: Processed Spark DataFrame ready for sentiment analysis.
    """
    # Read Excel file using pandas
    pandas_df = pd.read_excel(file_path)

    # Remove emojis from the text column
    pandas_df['text'] = pandas_df['text'].apply(remove_emojis)

    # Convert pandas DataFrame to Spark DataFrame
    df = spark.createDataFrame(pandas_df)

    # Remove duplicate rows
    df = df.dropDuplicates()

    df = clean_text(df)
    df = preprocess_text(df)
    return df

# Sentiment analysis and evaluation
def apply_vader_sentiment(df):
    """
    Applies the VADER sentiment analysis UDF to classify text sentiment.

    Args:
        df (DataFrame): Spark DataFrame with filtered tokens.

    Returns:
        DataFrame: Spark DataFrame with an additional column for predicted sentiment.
    """
    return df.withColumn("vader_predicted_label", vader_sentiment_udf(col("filtered_tokens")))

In [ ]:
# Main analysis process
file_path = "/content/tweets_stream_final1.xlsx"
df_processed = process_batch_data(file_path)
df_with_vader = apply_vader_sentiment(df_processed)

# Only keep the needed column
df_with_vader = df_with_vader.select("filtered_tokens", "vader_predicted_label")

# Since this is a test dataset without labels, we skip the accuracy calculation
df_with_vader.show(20, truncate=False)

In [ ]:
# Real-time plotting function
def plot_sentiment_analysis(aggregated_data):
    """
    Generates and updates a real-time Plotly chart to display sentiment analysis results.

    Args:
        aggregated_data (list): List of dictionaries containing sentiment percentages
                                and timestamps for visualization.
    """
    plot_times, positive_percentages, negative_percentages = [], [], []
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Positive Sentiment (%)', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Negative Sentiment (%)', line=dict(color='red')))
    fig.update_layout(title='Sentiment Analysis Over Time', xaxis_title='Time', yaxis_title='Sentiment Percentage', template='plotly_white')

    for entry in aggregated_data:
        plot_times.append(entry['time'])
        positive_percentages.append(entry['positive_percentage'])
        negative_percentages.append(entry['negative_percentage'])

        fig.data[0].x = plot_times
        fig.data[0].y = positive_percentages
        fig.data[1].x = plot_times
        fig.data[1].y = negative_percentages

        clear_output(wait=True)
        display(fig)

In [ ]:
# Simulated streaming processing (5 minutes interval)
def simulated_streaming_processing(data, chunk_size, output_path, window_size="1 minutes", toPlot=True, toAggregate=False):
    """
    Processes data in a simulated streaming fashion. Aggregates sentiment analysis
    results in time windows and optionally generates real-time plots.

    Args:
        data (DataFrame): Pandas DataFrame containing input data.
        chunk_size (int): Number of rows to process per chunk.
        output_path (str): Path to store aggregated results as a CSV file.
        window_size (str): Time window size for aggregation.
        toPlot (bool): If True, plots results in real-time.
        toAggregate (bool): If True, displays aggregated results.
    """
    start_time = pd.to_datetime("2024-11-30 20:20:00")
    end_time = start_time + pd.Timedelta(minutes=1)
    aggregated_results = []

    for i in range(0, len(data), chunk_size):
        # Extract chunk
        chunk = data.iloc[i:i + chunk_size]
        chunk['timestamp'] = start_time
        start_time = end_time
        end_time = start_time + pd.Timedelta(minutes=1)

        # Process data chunk
        chunk_spark = spark.createDataFrame(chunk)
        chunk_spark = clean_text(chunk_spark)
        chunk_spark = preprocess_text(chunk_spark)

        # Apply sentiment analysis
        df_with_vader = apply_vader_sentiment(chunk_spark)
        df_with_vader = df_with_vader.withColumn("processing_time", current_timestamp())

        # Aggregate in 5-minute windows
        df_aggregated = (
            df_with_vader
            .groupBy(window(col("processing_time"), window_size))
            .agg(
                count(when(col("vader_predicted_label") == 1.0, True)).alias("positive_count"),
                count(when(col("vader_predicted_label") == 0.0, True)).alias("negative_count"),
                count("*").alias("total_count")
            )
        )

        # Calculate percentages
        df_aggregated_with_percentages = df_aggregated.withColumn(
            "positive_percentage", round((col("positive_count") / col("total_count")) * 100, 1)
        ).withColumn(
            "negative_percentage", round((col("negative_count") / col("total_count")) * 100, 1)
        )

        df_aggregated_with_times = df_aggregated_with_percentages.select(
            "window.start", "window.end", "positive_count", "negative_count",
            "total_count", "positive_percentage", "negative_percentage"
        )

        # Convert to Pandas DataFrame for storage or visualization
        pandas_df = df_aggregated_with_times.toPandas()

        if os.path.exists(output_path):
            pandas_df.to_csv(output_path, mode='a', header=False, index=False)
        else:
            pandas_df.to_csv(output_path, mode='w', header=True, index=False)

        # Append results for plotting
        for _, row in pandas_df.iterrows():
            aggregated_results.append({
                'time': row['start'],
                'positive_percentage': row['positive_percentage'],
                'negative_percentage': row['negative_percentage']
            })

        # Whether to plot or aggregate (one at a time, since we're using cloud-based notebook)
        if toAggregate:
          df_aggregated_with_percentages.show(truncate=False)
        elif toPlot:
          plot_sentiment_analysis(aggregated_results)

        # Delay of 5 minutes to simulate real-time
        sleep(60)

def clean_output_folder(output_folder):
    """
    Cleans the output folder by removing all files.

    Args:
        output_folder (str): Path to the folder to be cleaned.
    """
    files = glob.glob(f"{output_folder}/*")
    for file in files:
        os.remove(file)
        print(f"Deleted {file}")

In [ ]:
clean_output_folder("output")
pandas_df = pd.read_excel(file_path) # Read .xlsx file
data = pd.DataFrame(pandas_df) # Convert to pandas DataFrame
output_path = "output/sentiment_results.csv"
# Aggregate the streaming data results
simulated_streaming_processing(data, chunk_size=100, output_path=output_path, toPlot=False, toAggregate=True)

In [ ]:
clean_output_folder("output")
pandas_df = pd.read_excel(file_path)
data = pd.DataFrame(pandas_df)
output_path = "output/sentiment_results.csv"
# Plot the streaming data results
simulated_streaming_processing(data, chunk_size=100, output_path=output_path, toPlot=True, toAggregate=False)

In [ ]:
# Stop Spark session
spark.stop()